In [7]:
import numpy as np

In [8]:
def hex2int(hexStr):
    # Up thành chữ viết hoa
    hexStr = hexStr.upper()
    # mapping
    mp = {'0': 0, '1': 1, '2': 2, '3': 3, 
          '4': 4, '5': 5, '6': 6, '7': 7,
          '8': 8, '9': 9, 'A': 10, 'B': 11,
          'C': 12, 'D': 13, 'E': 14, 'F': 15}
    intArr = []
    for i in range(len(hexStr)):
        intArr.append(mp[hexStr[i]])
    return intArr

def int2hex(intArr):
    # mapping
    mp = {0 :'0', 1 :'1', 2 :'2', 3 :'3', 
          4 :'4', 5 :'5', 6 :'6', 7 :'7',
          8 :'8', 9 :'9', 10 :'A', 11 :'B',
          12 :'C', 13 :'D', 14 :'E', 15 :'F'}
    hexStr = ''
    for i in range(len(intArr)):
        hexStr = hexStr + mp[intArr[i]]
    return hexStr

In [9]:
sbox = [12, 5, 6, 11, 9, 0, 10, 13, 3, 14, 15, 8, 4, 7, 1, 2]
MixColMatrix = [[4,  1, 2, 2],
                [8,  6, 5, 6],
                [11,14,10, 9],
                [2,  2,15,11]]
LED = 64
def FieldMult(a, b):
    ReductionPoly = 0x3
    x = a
    ret = 0
    for i in range(4):
        if ((b>>i)&1):
            ret ^= x
        if (x&0x8):
            x<<=1;
            x^= ReductionPoly
        else:
            x<<=1
    return ret&0xF

def AddKey(state, keyBytes, step):
    for i in range(4):
        for j in range(4):
            state[i][j] ^= keyBytes[int((4*i+j+step*16)%(LED/4))]
    return state

def AddConstants(state, r):
    rc = [0x01, 0x03, 0x07, 0x0F, 0x1F, 0x3E, 0x3D, 0x3B, 0x37, 0x2F,
      0x1E, 0x3C, 0x39, 0x33, 0x27, 0x0E, 0x1D, 0x3A, 0x35, 0x2B,
      0x16, 0x2C, 0x18, 0x30, 0x21, 0x02, 0x05, 0x0B, 0x17, 0x2E,
      0x1C, 0x38, 0x31, 0x23, 0x06, 0x0D, 0x1B, 0x36, 0x2D, 0x1A,
      0x34, 0x29, 0x12, 0x24, 0x08, 0x11, 0x22, 0x04]
    state[1][0] ^= 1
    state[2][0] ^= 2
    state[3][0] ^= 3
    
    state[0][0] ^= (LED>>4)&0xf
    state[1][0] ^= (LED>>4)&0xf
    state[2][0] ^= LED & 0xf
    state[3][0] ^= LED & 0xf
    
    tmp = (rc[r]>>3)&7
    state[0][1] ^= tmp
    state[2][1] ^= tmp
    tmp =  rc[r]&7
    state[1][1] ^= tmp
    state[3][1] ^= tmp
    
    return state

def SubCell(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = sbox[state[i][j]]
    return state

def ShiftRow(state):
    tmp = [0]*4
    for i in range(4):
        for j in range(4):
            tmp[j] = state[i][j]
        for j in range(4):
            state[i][j] = tmp[(j+i)%4]
    return state

def MixColumn(state):
    tmp = [0]*4
    for j in range(4):
        for i in range(4):
            sum = 0
            for k in range(4):
                sum^= FieldMult(MixColMatrix[i][k], state[k][j])
            tmp[i] = sum
        for i in range(4):
            state[i][j] = tmp[i]
    return state

def LEDencrypt(plainText, key):
    # đưa hex về dạng dãy các số
    plainText = hex2int(plainText)
    key = hex2int(key)
    
    # lập ma trận trạng thái
    state = np.reshape(np.array(plainText), (4, 4))
    
    # bắt đầu thuật toán
    state = AddKey(state, key, 0)
    for i in range(8):
        for j in range(4):
            state = AddConstants(state, i*4+j)
            state = SubCell(state)
            state = ShiftRow(state)
            state = MixColumn(state)
        state = AddKey(state, key, i+1)
    
    cipherText = ''
    for i in range(4):
        cipherText = cipherText + int2hex(state[i])
    return cipherText

In [10]:
# Main
key = 'f8a41b13b5ca4ee8'
plainText = '0be11a1c7f23f829'

cipherText = LEDencrypt(plainText, key)
print('Cipher Text:', cipherText)

Cipher Text: 86FAEDC807949F9F


In [11]:
key = '0123456789ABCDEF'
plainText = '0123456789ABCDEF'

cipherText = LEDencrypt(plainText, key)
print('Cipher Text:', cipherText)

Cipher Text: A003551E3893FC58


In [12]:
key = '0000000000000000'
plainText = '0000000000000000'

cipherText = LEDencrypt(plainText, key)
print('Cipher Text:', cipherText)

Cipher Text: 39C2401003A0C798
